# AWS RDS MySQL

In [3]:
import mysql.connector
import os
import pandas as pd
import yaml
from tqdm import tqdm

In [4]:
with open(r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\api\bot_world_config\database-config.yaml", "r") as f:
    try:
        config = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)
with open(r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\api\bot_world_config\seller-config.yaml", "r") as f:
    try:
        config.update(yaml.safe_load(f))
    except yaml.YAMLError as exc:
        print(exc)

In [22]:
cnx = mysql.connector.connect(user=config['DATABASE_USER'], password=config['DATABASE_PASSWORD'],
                              host=config['DATABASE_HOST'], database = config['DATABASE_NAME'],
                              port = 3306)
cursor = cnx.cursor()

In [40]:
# cursor.execute(f"CREATE DATABASE {config['DATABASE_NAME']};")
# cursor.execute("SHOW DATABASES;")
# cursor.execute("DROP database aws_bot_world_mysql;")

### create and import table

In [42]:
data = pd.read_csv(r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\shopee-20221117T041452Z-001\shopee\arietes_product_info.csv")
list_of_records = data.to_records(index = False).tolist()

In [43]:
data.head() 

,product_id,product_name,categories,image_url
0,23826146098,Ultrasonic Aroma Humidifier Led light changing,101129 - Home & Living/Home Fragrance & Aromat...,https://cf.shopee.sg/file/sg-11134207-23020-9n...
1,23821255327,Heavy Duty Canvas Tote Bags Stitching Contrast...,100093 - Women Bags/Tote Bags,https://cf.shopee.sg/file/sg-11134207-23020-yu...
2,23821254972,Hot Products EMS Vibration RF V Shape Other Ho...,101661 - Beauty/Beauty Tools/Facial Care Tools...,https://cf.shopee.sg/file/sg-11134207-23020-k1...
3,23821143235,Wart Removal Cream Natural Herbal Skin Care Re...,100427 - Health/Medical Supplies/First Aid Sup...,https://cf.shopee.sg/file/sg-11134207-23020-1u...
4,23720393304,Cute Kids Bag Cartoon Schoolbag Shoulder Bag S...,101744 - Baby & Kids Fashion/Baby & Kids Acces...,https://cf.shopee.sg/file/sg-11134207-23010-t7...


In [20]:
# cursor.execute(f"""DELETE FROM {config['TABLE_NAME']};""")
# cursor.execute(f"""DROP TABLE {config['TABLE_NAME']};""")

In [46]:
cursor.execute(
    f"""
    CREATE TABLE IF NOT EXISTS {config['PRODUCT_INFO_TABLE_NAME']} (
    product_id BIGINT PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    categories VARCHAR(255) NOT NULL,
    image_url VARCHAR(255) NOT NULL
    );
    """
)

In [48]:
for record in tqdm(list_of_records):
    try:
        cursor.execute(f"""INSERT INTO {config['PRODUCT_INFO_TABLE_NAME']} (product_id, product_name, categories, image_url) VALUES (%s,%s,%s,%s)""", record)
    except mysql.connector.Error as error:
        print(f"Failed to insert record {record}: {error}")  
cnx.commit()

100%|███████████████████████████████████████████████████████████████████████████████| 349/349 [00:02<00:00, 132.84it/s]


### read contents

In [48]:
cursor.execute(f"select * from {config['PRODUCT_INFO_TABLE_NAME']} limit 1;")

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [49]:
from pydantic import BaseModel
from typing import Optional

"""
A Product Recommendation
"""
class Product(BaseModel):
    product_id: int
    product_name: str
    categories: str
    image_url: str
Product(product_id = 23826146098, product_name = "a product name", categories = "a product category", item_url = "https://cf.shopee.sg/file/sg-11134207-23020-9nqha14zvsnvf4")

ValidationError: 1 validation error for Product
image_url
  field required (type=value_error.missing)

In [31]:
item_ids = tuple([2952217870, 0])

In [27]:
f"select * from {config['PRODUCT_INFO_TABLE_NAME']} where product_id in {};"

'select * from product_info where product_id in (2952217870, 0);'

In [32]:
cursor.execute(f"select * from {config['PRODUCT_INFO_TABLE_NAME']} where product_id in {item_ids};")

In [17]:
def establish_database_connection(config):
    cnx = mysql.connector.connect(user=config['DATABASE_USER'], password=config['DATABASE_PASSWORD'],
                                  host=config['DATABASE_HOST'], database = config['DATABASE_NAME'],
                                  port = 3306)
    return cnx

"""
Get info of 1 item 
"""
def get_product(cur, item_id, config):
    cur.execute(f"select * from {config['PRODUCT_INFO_TABLE_NAME']} where product_id={item_id};")
    item = next(cur)
    return Product(product_id = item[0], product_name = item[1], categories = item[2], image_url = item[3])


In [18]:
con = establish_database_connection(config)

In [19]:
cursor = con.cursor()

In [47]:
next(cursor)

StopIteration: 

In [48]:
cursor.execute()

In [46]:
next(cursor)

StopIteration: 

In [79]:
# cursor.close()
# cnx.close()

InternalError: Unread result found

# SQLITE3 - Local Database

In [23]:
import sqlite3
import pandas as pd

In [24]:
DESIRED_COLUMNS = ['product_id', 'product_name', 'categories', 'image_url']
DATABASE_DIR = r"C:\Users\tanch\Desktop\Bot.World\Bot.World\src\main\database\arietes"
DATABASE_NAME = "arietes"
PRODUCT_INFO_TABLE_NAME = "product_info"

## read data

In [4]:
data = pd.read_csv(r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\shopee-20221117T041452Z-001\shopee\arietes_product_info.csv")
list_of_records = data[DESIRED_COLUMNS].to_records(index = False).tolist()

In [5]:
data.head()

,product_id,product_name,categories,image_url
0,23826146098,Ultrasonic Aroma Humidifier Led light changing,101129 - Home & Living/Home Fragrance & Aromat...,https://cf.shopee.sg/file/sg-11134207-23020-9n...
1,23821255327,Heavy Duty Canvas Tote Bags Stitching Contrast...,100093 - Women Bags/Tote Bags,https://cf.shopee.sg/file/sg-11134207-23020-yu...
2,23821254972,Hot Products EMS Vibration RF V Shape Other Ho...,101661 - Beauty/Beauty Tools/Facial Care Tools...,https://cf.shopee.sg/file/sg-11134207-23020-k1...
3,23821143235,Wart Removal Cream Natural Herbal Skin Care Re...,100427 - Health/Medical Supplies/First Aid Sup...,https://cf.shopee.sg/file/sg-11134207-23020-1u...
4,23720393304,Cute Kids Bag Cartoon Schoolbag Shoulder Bag S...,101744 - Baby & Kids Fashion/Baby & Kids Acces...,https://cf.shopee.sg/file/sg-11134207-23010-t7...


## Create database
- create table, insert values

In [25]:
con = sqlite3.connect(f"{DATABASE_DIR}/{DATABASE_NAME}.db")
cur = con.cursor()

In [16]:
# cur.execute(f"""CREATE TABLE IF NOT EXISTS {PRODUCT_INFO_TABLE_NAME}(
#                 product_id INTEGER NOT NULL PRIMARY KEY, 
#                 product_name varchar(255), 
#                 categories varchar(255), 
#                 image_url varchar(255));""")  

In [17]:
# con.execute('ALTER TABLE arietes RENAME TO product_info')

In [8]:
# need to commit sqlite changes
# cur.executemany(f'INSERT INTO {PRODUCT_INFO_TABLE_NAME} VALUES(?,?,?,?)', list_of_records)
# con.commit()

In [26]:
cur.execute(f"""select * from {PRODUCT_INFO_TABLE_NAME}""")

In [31]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [32]:
cur.fetchall()

[('product_info',)]

In [27]:
remaining = cur.fetchall()

In [28]:
remaining

[(2952217870,
  'Ultra light Mini Umbrella 5 folding capsule windproof rain umbrella Very Small UV Sun Umbrella Pocket Mini Umbrella Wome',
  '100035 - Fashion Accessories/Accessories Sets & Packages',
  'https://cf.shopee.sg/file/3ac5329be0f139f7a7eb963cecfa2bab'),
 (3233299909,
  'Sport Face Mask with Activated Carbon Pm 2.5 reusable',
  '100128 - Health/Medical Supplies/Medical Gloves & Masks',
  'https://cf.shopee.sg/file/84930df960f41efc7afc9ba1b5a86e40'),
 (3295353571,
  'Carpal Tunnel Wrist Brace Support - Wrist Splint Arm Stabilizer & Hand Brace for Carpal Tunnel Syndrome Pain Relief',
  '100432 - Health/Medical Supplies/Injury & Disability Support/Braces & Supports',
  'https://cf.shopee.sg/file/d261a9b88a65ef8a108b1aaaf744d31c'),
 (3295377541,
  'Magnetic Therapy Self-Heating Arm Elbow Brace Support Belt Tourmaline Pain Relief Slimming Weight Loss Strap Bandage Arm',
  '100432 - Health/Medical Supplies/Injury & Disability Support/Braces & Supports',
  'https://cf.shopee.sg/fi

In [22]:
# con.commit()
# con.close()

In [10]:
# cur.fetchone()